<a href="https://colab.research.google.com/github/Hstlrr/hadith_variant_search/blob/main/Usingexcelforvariantsearching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install camel-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for camel-kenlm: filename=camel_kenlm-2023.3.17.2-cp310-cp310-linux_x86_64.whl size=3453089 sha256=b050ea653163890f8b3d3422efa466c4f3f8a714ee3095fbcd9179aeb4409594
  Stored in directory: /root/.cache/pip/wheels/29/c5/32/09633c3b70fdfc470b2fb912bd9e90d8d6814df68c794dcaa6
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a631af7eedd1db7e64df68ccb722a9fc956b

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.normalize import normalize_unicode, normalize_alef_maksura_ar, normalize_alef_ar, normalize_teh_marbuta_ar


In [ ]:
dataset_path = '/content/combined_hadiths.csv'

In [ ]:
df = pd.read_csv(dataset_path)

<ipython-input-4-987f6916f0f8>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dataset_path)


In [ ]:
matn_texts = df['Arabic_Matn'].tolist()

df = df.dropna(subset=['Arabic_Matn'])
matn_texts = df['Arabic_Matn'].tolist()

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=simple_word_tokenize)
tfidf_matrix = vectorizer.fit_transform(matn_texts)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
def find_and_display_variants(input_text, threshold=0.8):
    # Clean and transform the input text
    cleaned_input = input_text.replace('\u200f', '')
    input_vector = vectorizer.transform([cleaned_input])

    # Compute the cosine similarity
    cosine_sim_input = cosine_similarity(input_vector, tfidf_matrix)

    # Find similar hadiths
    similarity_scores = list(enumerate(cosine_sim_input[0]))
    similar_hadiths_indices = [i for i, score in similarity_scores if score > threshold]

    print("Variants of the input hadith with at least 80% similarity:")
    for index in similar_hadiths_indices:
        chapter_number = df['Chapter_Number'][index]
        hadith_number = df['Hadith_number'][index]
        text = df['Arabic_Matn'][index]
        collection = df['Collection'][index]
        print(f"Collection: {collection}, Index: {index}, Chapter: {chapter_number}, Hadith: {hadith_number}, Similarity: {cosine_sim_input[0][index]}, Text: {text}")



In [ ]:
input_hadith = 'أَنَّ رَسُولَ اللَّهِ صلى الله عليه وسلم أَمَرَ بِقَتْلِ الْكِلاَبِ ثُمَّ قَالَ مَا لَهُمْ وَلِلْكِلاَبِ ثُمَّ رَخَّصَ لَهُمْ فِي كَلْبِ الصَّيْدِ'
# Test the function with the input hadith
find_and_display_variants(input_hadith, threshold=0.3)

Variants of the input hadith with at least 80% similarity:
Collection: AbuDaud, Index: 73, Chapter: 1, Hadith: 74.0, Similarity: 0.4705217539631458, Text: أَنَّ رَسُولَ اللَّهِ صلى الله عليه وسلم أَمَرَ بِقَتْلِ الْكِلاَبِ ثُمَّ قَالَ مَا لَهُمْ وَلَهَا فَرَخَّصَ فِي كَلْبِ الصَّيْدِ وَفِي كَلْبِ الْغَنَمِ وَقَالَ إِذَا وَلَغَ الْكَلْبُ فِي الإِنَاءِ فَاغْسِلُوهُ سَبْعَ مِرَارٍ وَالثَّامِنَةُ عَفِّرُوهُ بِالتُّرَابِ قَالَ أَبُو دَاوُدَ وَهَكَذَا قَالَ ابْنُ مُغَفَّلٍ
Collection: Bukhari, Index: 8257, Chapter: 59, Hadith: 3199.0, Similarity: 0.5164663838080821, Text: قَالَ النَّبِيُّ صلى الله عليه وسلم لأَبِي ذَرٍّ حِينَ غَرَبَتِ الشَّمْسُ ‏"‏ تَدْرِي أَيْنَ تَذْهَبُ ‏"‏‏.‏ قُلْتُ اللَّهُ وَرَسُولُهُ أَعْلَمُ‏.‏ قَالَ ‏"‏ فَإِنَّهَا تَذْهَبُ حَتَّى تَسْجُدَ تَحْتَ الْعَرْشِ، فَتَسْتَأْذِنَ فَيُؤْذَنَ لَهَا، وَيُوشِكُ أَنْ تَسْجُدَ فَلاَ يُقْبَلَ مِنْهَا، وَتَسْتَأْذِنَ فَلاَ يُؤْذَنَ لَهَا، يُقَالُ لَهَا ارْجِعِي مِنْ حَيْثُ جِئْتِ‏.‏ فَتَطْلُعُ مِنْ مَغْرِبِهَا، فَذَلِكَ قَوْلُهُ تَعَا